In [3]:
import cv2
import pytesseract
import fitz 
from PIL import Image
from inference_sdk import InferenceHTTPClient


In [4]:
pip install pymupdf



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:

def pdf_to_jpg(pdf_path):
    pdf_document = fitz.open(pdf_path)
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        output_path = f"cv_{page_number + 1}.jpg"
        img.save(output_path, "JPEG")
        return output_path
    



In [56]:
import re 
def get_mail(path):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    pdf_document = fitz.open(path, filetype="pdf")
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    liste_emails = r.findall(text) 
    emails_valides = []
    for email in liste_emails:
        if email.endswith(('.com', '.tn','.org','.net')):
            emails_valides.append(email)
    return emails_valides[0]


In [59]:
print(get_mail('cv_hajri.pdf'))

hajriimen154@gmail.com


In [42]:
def get_skills_cv(path):
    image_path = pdf_to_jpg(path)
    CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="BIrE2FC1DoCP2tdXB9MA"
    )
    
    # Define the model ID
    model_id = "resume-parser-bchlq/1"
    result = CLIENT.infer(image_path, model_id=model_id)
    
    image = cv2.imread(image_path)

    # Extract the 'Skills' section from the result
    skills_section = None
    for prediction in result['predictions']:
        if prediction['class'] == 'skills':
            x = int(prediction['x'] - prediction['width'] / 2)
            y = int(prediction['y'] - prediction['height'] / 2)
            width = int(prediction['width'])
            height = int(prediction['height'])
            
            skills_image = image[y:y+height, x:x+width]            
            skills_image_rgb = cv2.cvtColor(skills_image, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(skills_image_rgb)
            skills_text = pytesseract.image_to_string(pil_image)
            
            return skills_text
    else:
        print("No 'Skills' section found in the image.")


In [43]:
x=get_skills_cv('cv3.jpeg')

In [44]:
print(x)

 

COMPETENCES

+ Langages de programmation :
HTML, CSS, Tailwind CSS,
JavaScript, React Js, Node Js,
Flutter, PHP, Laravel, Java

* Bases de données : MySQL,
MongoDB, Firebase

* Logiciels : Microsoft Office Word,
Visual Studio Code, Android Studio,

Postman, Power Bl, Figma



In [14]:
from sentence_transformers import SentenceTransformer, util

sentences = ["html css", "HTML CSS"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')




In [15]:
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)

tensor([[1.0000]])

In [68]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# SMTP server configuration
smtp_server = 'smtp.gmail.com'
smtp_port = 465
smtp_username = 'amyne095@gmail.com'  # Use your full email address
smtp_password = 'kdbm reqo pfti tfnu'     # Use your app-specific password if 2FA is enabled

# Email details
to_email = 'hassounibarka@gmail.com'
from_email = smtp_username
subject = 'Hello, world!'
body = 'This is a test email.'

# Create the email message
message = MIMEMultipart()
message['From'] = from_email
message['To'] = to_email
message['Subject'] = subject
message.attach(MIMEText(body, 'plain'))

# Send the email
try:
    with smtplib.SMTP(smtp_server, smtp_port) as smtp:
        smtp.starttls()  # Secure the connection
        smtp.login(smtp_username, smtp_password)
        smtp.send_message(message)
        print(f"Email sent to {to_email} with subject '{subject}'")
except Exception as e:
    print(f"Failed to send email: {e}")


Failed to send email: Connection unexpectedly closed


In [70]:
import smtplib
import ssl
from email.message import EmailMessage

email_password ='kdbm reqo pfti tfnu'
email_sender = 'amyne095@gmail.com'
email_receiver = 'hassounibarka@gmail.com'

In [72]:
def send_mail(email_sender, email_password,email_receiver):
    # Set the subject and body of the email
    subject = 'Merci pour votre participation au processus de recrutement'
    body = """
    Bonjour,

    Nous vous remercions sincèrement pour votre participation au processus de recrutement pour le poste de [Nom du poste] au sein de notre entreprise. Nous avons bien reçu votre candidature et tenons à vous exprimer notre gratitude pour l'intérêt que vous portez à [Nom de l'entreprise].

    Votre dossier est actuellement en cours d'évaluation, et nous ne manquerons pas de vous tenir informé(e) de la suite des événements dans les meilleurs délais.

    En attendant, nous vous souhaitons une excellente journée.

    Bien cordialement,
    """
    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(body)

    # Add SSL (layer of security)
    context = ssl.create_default_context()

    # Log in and send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())

In [73]:
send_mail(email_sender,email_password,email_receiver)